In [59]:
import cv2
import os
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [57]:
def get_frames(path, fps=None):
    cap = cv2.VideoCapture(path)

    time_increment = (
        1/fps 
        if fps is not None 
        else None
    )

    res = []
    success = 1
    sec = 0
    while success:
        if time_increment is not None:
            sec += time_increment
            cap.set(cv2.CAP_PROP_POS_MSEC, 1000*sec)
        success, image = cap.read()
        if success:
            res.append(image)

    return res


def save_frames(
    frames, 
    dir, 
    prefix="frame"
):
    for i, frame in enumerate(frames):
        cv2.imwrite(str(osp.join(dir, f"{prefix}_{i}.jpg")), frame)


def to_gray(frames):
    res = [cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in frames]
    return res


def to_scale(frames, scale=0.5):
    res = []
    for f in frames:
        width = int(f.shape[1] * scale)
        height = int(f.shape[0] * scale)
        dim = (width, height)
        resized = cv2.resize(f, dim, interpolation=cv2.INTER_AREA)
        res.append(resized)
        
    return res


def read_pickle(dir):
    with open(dir, 'rb') as handle:
        b = pickle.load(handle)
    return b


def write_pickle(dir, data):
    with open(dir, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

def to_pickle_name(file):
    return file.split(".")[0] + ".pkl"


def flatten(frames):
    return [f.flatten() for f in frames]

In [69]:
def to_fps_gray_scale(
    base_dir,
    save_dir,
    files,
    fps,
    scale,
):
    for file in files:
        path = osp.join(base_dir, file)
        res = get_frames(path, fps)
        gray = to_gray(res)
        resized_gray = to_scale(gray, scale)
        
        write_pickle(
            osp.join(save_dir, to_pickle_name(file)), 
            resized_gray
        )
        

def get_files(dir, format="mp4"):
    res = []
    for filename in os.listdir(dir):
        if filename.split(".")[-1] == format:
            res.append(filename)
    return res

In [ ]:
base_dir = "C:/Users/aphri/Documents/t0002/pycharm/data/action_dataset0003"
save_dir = "C:/Users/aphri/Documents/t0002/pycharm/data/ar_fps2_gray_scale3/pickle"

files = get_mp4_files(base_dir)
    
to_fps_gray_scale(
    base_dir=base_dir,
    save_dir=save_dir,
    files=files,
    fps=2,
    scale=0.3,
)

In [ ]:
def to_actions(f):
    words = f.split("_")
    res = []
    for w in words:
        if w.startswith("y") and w[1:].isnumeric():
            break
        res.append(w)
    
    return res
    

In [ ]:
pickle_files = [
    "backhandspring_backhandspring_backhandspring_y0_x0.pkl",
    "backhandspring_backhandspring_clap_y45_x45.pkl",
]

res = []
for f in pickle_files:
    frames = read_pickle(osp.join(save_dir, f))
    frames = flatten(frames)
    actions = to_actions(f)
    count = len(actions)
    
    res.append((frames, actions, count))
    
    

In [ ]:
" ".join("a", "b")